## Modelo datos estructurados
Este notebook desarrolla un primer modelo para resolver el problema de Petfinder. Empezamos haciendo un modelo inicial muy simple para ver la viabilidad de resolver el problema. Luego analizamos como se comporta la métrica kappa propuesta y vemos la matriz de confusión. Finalmente hacemos una optimizacin de hiperparametros evaluando con train/test y otra validando con 5 fold CV y testeando en el 20% de los datos

In [1]:
#Import de librerias basicas tablas y matrices
import numpy as np 
import pandas as pd 

#Gradient Boosting
import lightgbm as lgb

#Funciones auxiliares sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold #Split y cross Validation
from sklearn.metrics import cohen_kappa_score, accuracy_score, balanced_accuracy_score #Metricas
from sklearn.utils import shuffle 

#Visualizacióon
from plotly import express as px

#Plot de matriz de confusion normalizada en actuals
from utils import plot_confusion_matrix

import os

#Optimizacion de hiperparametros
import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

#Guardado de objetos en archivos joblib
from joblib import load, dump


c:\Users\cacic\anaconda3\envs\ldi2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Paths para acceso archivos
#Este notebook asume la siguiente estructura de carpetas a partir de la ubicacion de base_dir 
#(dos niveles arriba de la carpeta donde se ejecuta el notebook). 
# /UA_MDM_LDI_II/
# /UA_MDM_LDI_II/input
# /UA_MDM_LDI_II/input/petfinder-adoption-prediction/            <- Aca deben ir todos los archivos de datos de la competencia 
# /UA_MDM_LDI_II/tutoriales/                       <- Aca deben poner los notebooks y scripts que les compartimos
# /UA_MDM_LDI_II/work/                             <- Resultados de notebooks iran dentro de esta carpeta en subcarpetas
# /UA_MDM_LDI_II/work/models/                     <- Modelos entrenados en archivos joblibs
# /UA_MDM_LDI_II/work/optuna_temp_artifacts/      <- Archivos que queremos dejar como artefacto de un trial de optuna (optuna los copiara a la carpeta de abajo)
# /UA_MDM_LDI_II/work/optuna_artifacts/           <- Archivos con artefactos que sibimos a optuna

#Subimos dos niveles para quedar en la carpeta que contiene input y UA_MDM_LDI_II
BASE_DIR = '../'

#Datos de entrenamiento 
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train.csv")

#Salida de modelos entrenados
PATH_TO_MODELS = os.path.join(BASE_DIR, "work/models")

#Artefactos a subir a optuna
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "work/optuna_temp_artifacts")

#Artefactos que optuna gestiona
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "work/optuna_artifacts")


SEED = 42 #Semilla de procesos aleatorios (para poder replicar exactamente al volver a correr un modelo)
TEST_SIZE = 0.2 #Facción para train/test= split

In [4]:
# Datos Tabulares
dataset = pd.read_csv(PATH_TO_TRAIN)

In [5]:
#Columnas del dataset
dataset.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

In [6]:
# Agregar la columna 'mixed' basada en las condiciones especificadas
dataset['Pura'] = np.where(
  (dataset['Breed1'].notna()) & (dataset['Breed2'].notna()), 'Pure', np.where(
    dataset['Breed1'] != 'Mixed Breed', 'Pure', 'Mixed'
   )
  )

In [7]:
dataset

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,Pura
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,Pure
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,Pure
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,Pure
3,1,Miko,4,307,0,2,1,2,0,2,...,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,Pure
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,Pure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,NaN,2,266,0,3,1,0,0,2,...,4,0,41326,61c84bd7bcb6fb31d2d480b1bcf9682e,0,I have 4 kittens that need to be adopt urgentl...,dc0935a84,3.0,2,Pure
14989,2,Serato & Eddie,60,265,264,3,1,4,7,2,...,2,0,41326,1d5096c4a5e159a3b750c5cfcf6ceabf,0,Serato(female cat- 3 color) is 4 years old and...,a01ab5b30,3.0,4,Pure
14990,2,Monkies,2,265,266,3,5,6,7,3,...,5,30,41326,6f40a7acfad5cc0bb3e44591ea446c05,0,"Mix breed, good temperament kittens. Love huma...",d981b6395,5.0,3,Pure
14991,2,Ms Daym,9,266,0,2,4,7,0,1,...,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4,Pure


In [8]:
#Feature Engeeneiring
dataset1 = pd.get_dummies(dataset, columns=["Pura"], dtype=int)

In [9]:
dataset1

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,Pura_Pure
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,1
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,1
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,1
3,1,Miko,4,307,0,2,1,2,0,2,...,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,1
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,NaN,2,266,0,3,1,0,0,2,...,4,0,41326,61c84bd7bcb6fb31d2d480b1bcf9682e,0,I have 4 kittens that need to be adopt urgentl...,dc0935a84,3.0,2,1
14989,2,Serato & Eddie,60,265,264,3,1,4,7,2,...,2,0,41326,1d5096c4a5e159a3b750c5cfcf6ceabf,0,Serato(female cat- 3 color) is 4 years old and...,a01ab5b30,3.0,4,1
14990,2,Monkies,2,265,266,3,5,6,7,3,...,5,30,41326,6f40a7acfad5cc0bb3e44591ea446c05,0,"Mix breed, good temperament kittens. Love huma...",d981b6395,5.0,3,1
14991,2,Ms Daym,9,266,0,2,4,7,0,1,...,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4,1


In [10]:
#Feature Engeeneiring
dataset2 = pd.get_dummies(dataset1, columns=["Breed1"], dtype=int)

In [11]:
dataset2

,Type,Name,Age,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,...,Breed1_298,Breed1_299,Breed1_300,Breed1_301,Breed1_302,Breed1_303,Breed1_304,Breed1_305,Breed1_306,Breed1_307
0,2,Nibble,3,0,1,1,7,0,1,1,...,0,1,0,0,0,0,0,0,0,0
1,2,No Name Yet,1,0,1,1,2,0,2,2,...,0,0,0,0,0,0,0,0,0,0
2,1,Brisco,1,0,1,2,7,0,2,2,...,0,0,0,0,0,0,0,0,0,1
3,1,Miko,4,0,2,1,2,0,2,1,...,0,0,0,0,0,0,0,0,0,1
4,1,Hunter,1,0,1,1,0,0,2,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,NaN,2,0,3,1,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
14989,2,Serato & Eddie,60,264,3,1,4,7,2,2,...,0,0,0,0,0,0,0,0,0,0
14990,2,Monkies,2,266,3,5,6,7,3,2,...,0,0,0,0,0,0,0,0,0,0
14991,2,Ms Daym,9,0,2,4,7,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#Separo un 20% para test estratificado opr target
train, test = train_test_split(dataset2,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               stratify = dataset2.AdoptionSpeed)

In [13]:
#Armo listas con features de texto y numericas
char_feats = [f for f in dataset2.columns if dataset2[f].dtype=='O']
numeric_feats = [f for f in dataset2.columns if dataset2[f].dtype!='O']

In [14]:
#Lista de features numericas
numeric_feats

['Type',
 'Age',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Dewormed',
 'Sterilized',
 'Health',
 'Quantity',
 'Fee',
 'State',
 'VideoAmt',
 'PhotoAmt',
 'AdoptionSpeed',
 'Pura_Pure',
 'Breed1_0',
 'Breed1_1',
 'Breed1_3',
 'Breed1_5',
 'Breed1_7',
 'Breed1_10',
 'Breed1_11',
 'Breed1_15',
 'Breed1_16',
 'Breed1_17',
 'Breed1_18',
 'Breed1_19',
 'Breed1_20',
 'Breed1_21',
 'Breed1_23',
 'Breed1_24',
 'Breed1_25',
 'Breed1_26',
 'Breed1_31',
 'Breed1_32',
 'Breed1_39',
 'Breed1_42',
 'Breed1_44',
 'Breed1_49',
 'Breed1_50',
 'Breed1_56',
 'Breed1_58',
 'Breed1_60',
 'Breed1_61',
 'Breed1_64',
 'Breed1_65',
 'Breed1_69',
 'Breed1_70',
 'Breed1_71',
 'Breed1_72',
 'Breed1_75',
 'Breed1_76',
 'Breed1_78',
 'Breed1_81',
 'Breed1_82',
 'Breed1_83',
 'Breed1_85',
 'Breed1_88',
 'Breed1_93',
 'Breed1_97',
 'Breed1_98',
 'Breed1_99',
 'Breed1_100',
 'Breed1_102',
 'Breed1_103',
 'Breed1_105',
 'Breed1_108',
 'Breed1_109',
 'Breed1_11

In [15]:

#Defino features a usar en un primer modelo de prueba
features = ['Type',
 'Age',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Dewormed',
 'Sterilized',
 'Health',
 'Quantity',
 'Fee',
 'State',
 'VideoAmt',
 'PhotoAmt',
 'Pura_Pure',
 'Breed1_0',
 'Breed1_1',
 'Breed1_3',
 'Breed1_5',
 'Breed1_7',
 'Breed1_10',
 'Breed1_11',
 'Breed1_15',
 'Breed1_16',
 'Breed1_17',
 'Breed1_18',
 'Breed1_19',
 'Breed1_20',
 'Breed1_21',
 'Breed1_23',
 'Breed1_24',
 'Breed1_25',
 'Breed1_26',
 'Breed1_31',
 'Breed1_32',
 'Breed1_39',
 'Breed1_42',
 'Breed1_44',
 'Breed1_49',
 'Breed1_50',
 'Breed1_56',
 'Breed1_58',
 'Breed1_60',
 'Breed1_61',
 'Breed1_64',
 'Breed1_65',
 'Breed1_69',
 'Breed1_70',
 'Breed1_71',
 'Breed1_72',
 'Breed1_75',
 'Breed1_76',
 'Breed1_78',
 'Breed1_81',
 'Breed1_82',
 'Breed1_83',
 'Breed1_85',
 'Breed1_88',
 'Breed1_93',
 'Breed1_97',
 'Breed1_98',
 'Breed1_99',
 'Breed1_100',
 'Breed1_102',
 'Breed1_103',
 'Breed1_105',
 'Breed1_108',
 'Breed1_109',
 'Breed1_111',
 'Breed1_114',
 'Breed1_117',
 'Breed1_119',
 'Breed1_122',
 'Breed1_123',
 'Breed1_125',
 'Breed1_128',
 'Breed1_129',
 'Breed1_130',
 'Breed1_132',
 'Breed1_139',
 'Breed1_141',
 'Breed1_143',
 'Breed1_145',
 'Breed1_146',
 'Breed1_147',
 'Breed1_148',
 'Breed1_150',
 'Breed1_152',
 'Breed1_154',
 'Breed1_155',
 'Breed1_165',
 'Breed1_167',
 'Breed1_169',
 'Breed1_173',
 'Breed1_176',
 'Breed1_178',
 'Breed1_179',
 'Breed1_182',
 'Breed1_185',
 'Breed1_187',
 'Breed1_188',
 'Breed1_189',
 'Breed1_190',
 'Breed1_192',
 'Breed1_195',
 'Breed1_197',
 'Breed1_199',
 'Breed1_200',
 'Breed1_201',
 'Breed1_202',
 'Breed1_203',
 'Breed1_204',
 'Breed1_205',
 'Breed1_206',
 'Breed1_207',
 'Breed1_212',
 'Breed1_213',
 'Breed1_214',
 'Breed1_215',
 'Breed1_217',
 'Breed1_218',
 'Breed1_224',
 'Breed1_227',
 'Breed1_228',
 'Breed1_231',
 'Breed1_232',
 'Breed1_233',
 'Breed1_234',
 'Breed1_237',
 'Breed1_239',
 'Breed1_240',
 'Breed1_241',
 'Breed1_242',
 'Breed1_243',
 'Breed1_244',
 'Breed1_245',
 'Breed1_246',
 'Breed1_247',
 'Breed1_248',
 'Breed1_249',
 'Breed1_250',
 'Breed1_251',
 'Breed1_252',
 'Breed1_253',
 'Breed1_254',
 'Breed1_256',
 'Breed1_257',
 'Breed1_260',
 'Breed1_262',
 'Breed1_263',
 'Breed1_264',
 'Breed1_265',
 'Breed1_266',
 'Breed1_267',
 'Breed1_268',
 'Breed1_269',
 'Breed1_270',
 'Breed1_271',
 'Breed1_272',
 'Breed1_273',
 'Breed1_274',
 'Breed1_276',
 'Breed1_277',
 'Breed1_279',
 'Breed1_280',
 'Breed1_281',
 'Breed1_282',
 'Breed1_283',
 'Breed1_284',
 'Breed1_285',
 'Breed1_286',
 'Breed1_287',
 'Breed1_288',
 'Breed1_289',
 'Breed1_290',
 'Breed1_292',
 'Breed1_293',
 'Breed1_294',
 'Breed1_295',
 'Breed1_296',
 'Breed1_297',
 'Breed1_298',
 'Breed1_299',
 'Breed1_300',
 'Breed1_301',
 'Breed1_302',
 'Breed1_303',
 'Breed1_304',
 'Breed1_305',
 'Breed1_306',
 'Breed1_307']

label = 'AdoptionSpeed'

In [16]:
#Genero dataframes de train y test con sus respectivos targets
X_train = train[features]
y_train = train[label]

X_test = test[features]
y_test = test[label]

In [17]:
#Entreno un modelo inicial sin modificar hiperparametros. Solamente especifico el numero de clases y el tipo de modelo como clasificación
lgb_params = params = {
                        'objective': 'multiclass',
                        'num_class': len(y_train.unique())
                        }


#genero el objeto Dataset que debo pasarle a lightgbm para que entrene
lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)

#entreno el modelo con los parametros por defecto
lgb_model = lgb.train(lgb_params,
                      lgb_train_dataset)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003495 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 393
[LightGBM] [Info] Number of data points in the train set: 11994, number of used features: 55
[LightGBM] [Info] Start training from score -3.599148
[LightGBM] [Info] Start training from score -1.579379
[LightGBM] [Info] Start training from score -1.311924
[LightGBM] [Info] Start training from score -1.526206
[LightGBM] [Info] Start training from score -1.273359


In [18]:
#Obtengo las predicciones sobre el set de test. El modelo me da una lista de probabilidades para cada clase y tomo la clase con mayor probabilidad con la funcion argmax
y_pred = lgb_model.predict(X_test).argmax(axis=1)

#Calculo el Kappa
cohen_kappa_score(y_test,y_pred, weights = 'quadratic')

np.float64(0.31204440794051513)

In [19]:
#Muestro la matriz de confusión
display(plot_confusion_matrix(y_test,y_pred))

In [20]:
#Vamos a poner en perspectiva el score de Kappa


#Cual es el score perfecto? Evaluo la clase real contra si misma. Es decir, el caso en que el modelo establece todas las clases en su valor real
cohen_kappa_score(y_test,y_test, weights = 'quadratic')

np.float64(1.0)

In [21]:
#Como se veria la matriz de confusión
display(plot_confusion_matrix(y_test,y_test))

In [22]:
#Vamos a ver como se comporta kappa si hago una predicción al azar (respetando las proporciones de cada clase)
y_shuffled = shuffle(y_test,
                     random_state = 42)


#Genero diccionarios para cambiar algunas predicicones reales por una prediccion cercana y_cerca y una lejana y_lejos a la real 
# ejemplo: la real es 0, voy a estimar 1 para la cercana y 4 para la lejana
dict_map_cerca = {0:1,
                  1:2,
                  2:3,
                  3:4,
                  4:3}

dict_map_lejos = {0:4,
                  1:4,
                  2:0,
                  3:0,
                  4:0}

y_cerca = [dict_map_cerca[i] for i in y_test]

y_lejos = [dict_map_lejos[i] for i in y_test]


In [23]:

#Vamos a simular que la probabilidad de tener la prediccion real en casa muestra varia de 0 a 100. 
#Genero una numero aleatorio para cada muestra
random_list =  np.random.rand(len(y_test))

#inicializo un dataframe de resultados vacio
kappa_progression = pd.DataFrame()

#La variable i tiene un umbral para ir variando la cantidad de aciertos desde 0% a 100%
for i in range(101):

    #Genero la prediccion para i% de aciertos donde cuando no acierto me quedo con una prediccion al azar (podria ser la "correcta" pero solo por azar)
    y_simulado = [y_test.iloc[sample] if random_list[sample]<i/100 else y_shuffled.iloc[sample] for sample in range(len(y_test))]

    #Genero la prediccion para i% de aciertos donde cuando no acierto me quedo con una prediccion cercana o lejana a la correcta
    y_simulado_cerca = [y_test.iloc[sample] if random_list[sample]<i/100 else y_cerca[sample] for sample in range(len(y_test))]
    y_simulado_lejos = [y_test.iloc[sample] if random_list[sample]<i/100 else y_lejos[sample] for sample in range(len(y_test))]


    #Grabo los resultados en un dataframe para cada i% de aciertos
    kappa_progression = pd.concat([kappa_progression,
                                   pd.DataFrame({'Conocidos':[i],
                                                'kappa':cohen_kappa_score(y_test,
                                                                        y_simulado,
                                                                        weights = 'quadratic'),
                                                'kappa_cerca':cohen_kappa_score(y_test,
                                                                        y_simulado_cerca,
                                                                        weights = 'quadratic'),
                                                'kappa_lejos':cohen_kappa_score(y_test,
                                                                        y_simulado_lejos,
                                                                        weights = 'quadratic'),                                                                        
                                                'accuracy':accuracy_score(y_test,
                                                                        y_simulado),
                                                'balanced_accuracy':balanced_accuracy_score(y_test,
                                                                        y_simulado),
                                                                        })],
                ignore_index=True)

In [24]:
#Grafico el comportamiento de la métrica a medida que incremento los aciertos. Tambien muestro lor resultados de otras metricas como Accuracy y Balanced Accuracy
px.line(kappa_progression,x='Conocidos',y=['kappa',
                                           'kappa_cerca',
                                           'kappa_lejos',
                                           'accuracy',
                                           'balanced_accuracy'])

In [25]:

#A modo de ejemplo muestro kappa y matriz de confusion para 50% de aciertos donde los errores quedan cerca de la clase correcta
y_simulado_cerca = [y_test.iloc[sample] if random_list[sample]<50/100 else y_cerca[sample] for sample in range(len(y_test))]

display(plot_confusion_matrix(y_test,y_simulado_cerca, 
                              title = "Kappa " + str(cohen_kappa_score(y_test,y_simulado_cerca, weights = 'quadratic'))))



In [26]:

#A modo de ejemplo muestro kappa y matriz de confusion para 50% de aciertos donde los errores quedan lejos de la clase correcta
y_simulado_lejos = [y_test.iloc[sample] if random_list[sample]<50/100 else y_lejos[sample] for sample in range(len(y_test))]

display(plot_confusion_matrix(y_test,y_simulado_lejos, 
                              title = "Kappa " + str(cohen_kappa_score(y_test,y_simulado_lejos, weights = 'quadratic'))))


In [27]:
#Pruebo un modelo alternativo donde en vez de usar la version multiclass real de lightGBM utilizo One vs All

lgb_params = params = {
                        'objective': 'multiclassova',
                        'num_class': len(y_train.unique())
                        }


lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


lgb_model = lgb.train(lgb_params,
                      lgb_train_dataset)

[LightGBM] [Info] Number of positive: 328, number of negative: 11666
[LightGBM] [Info] Number of positive: 2472, number of negative: 9522
[LightGBM] [Info] Number of positive: 3230, number of negative: 8764
[LightGBM] [Info] Number of positive: 2607, number of negative: 9387
[LightGBM] [Info] Number of positive: 3357, number of negative: 8637
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003228 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 393
[LightGBM] [Info] Number of data points in the train set: 11994, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.027347 -> initscore=-3.571420
[LightGBM] [Info] Start training from score -3.571420
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.206103 -> initscore=-1.348577
[LightGBM] [Info] Start training from score -1.348577
[LightGBM] [Info] [binary:BoostFrom

In [28]:
#MAtriz de confusion y Kappa dfe OVA
y_pred = lgb_model.predict(X_test).argmax(axis=1)

display(plot_confusion_matrix(y_test,y_pred))

{'kappa':cohen_kappa_score(y_test,
                y_pred,
                weights = 'quadratic'),
 'accuracy':accuracy_score(y_test,y_pred),
 'balanced_accuracy':balanced_accuracy_score(y_test,y_pred)}




{'kappa': np.float64(0.3174951811730289),
 'accuracy': 0.3864621540513504,
 'balanced_accuracy': np.float64(0.3178739499851616)}

## Optimizacion de hiperparametros modelo train/test

In [29]:

#Funcion que vamos a optimizar. Optuna requiere que usemos el objeto trial para generar los parametros a optimizar
def lgb_objective(trial):
    #PArametros para LightGBM
    lgb_params = {      
                        #PArametros fijos
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique()),
                        #Hiperparametros a optimizar utilizando suggest_float o suggest_int segun el tipo de dato
                        #Se indica el nombre del parametro, valor minimo, valor maximo 
                        #en elgunos casos el parametro log=True para parametros que requieren buscar en esa escala
                        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        } 

    #Genero objeto dataset de entrenamiento
    lgb_train_dataset = lgb.Dataset(data=X_train,
                                    label=y_train)

    #ajuste de modelo
    lgb_model = lgb.train(lgb_params,
                        lgb_train_dataset)
    
    #Devuelvo el score en test
    return(cohen_kappa_score(y_test,lgb_model.predict(X_test).argmax(axis=1),
                             weights = 'quadratic'))

In [30]:
#Defino el estudio a optimizar
study = optuna.create_study(direction='maximize', #buscamos maximizar la metrica
                            storage="sqlite:///../work/db.sqlite3",  # Specify the storage URL here.
                            study_name="04 - LGB Multiclass", #nombre del experimento
                            load_if_exists=True) #continuar si ya existe

#Corremos 100 trials para buscar mejores parametros
study.optimize(lgb_objective, n_trials=100)

[I 2024-08-22 19:37:39,963] Using an existing study with name '04 - LGB Multiclass' instead of creating a new one.
[I 2024-08-22 19:37:44,024] Trial 400 finished with value: 0.29980276024066055 and parameters: {'lambda_l1': 3.2747914679433843e-06, 'lambda_l2': 0.9725526858293452, 'num_leaves': 36, 'feature_fraction': 0.8821521637862549, 'bagging_fraction': 0.521204734343403, 'bagging_freq': 6, 'min_child_samples': 91}. Best is trial 241 with value: 0.33769755881911834.
[I 2024-08-22 19:37:46,992] Trial 401 finished with value: 0.3000931841267974 and parameters: {'lambda_l1': 1.3654286143497509e-05, 'lambda_l2': 1.5859700506851437, 'num_leaves': 256, 'feature_fraction': 0.9115927664064756, 'bagging_fraction': 0.5152388184339238, 'bagging_freq': 6, 'min_child_samples': 90}. Best is trial 241 with value: 0.33769755881911834.
[I 2024-08-22 19:37:48,967] Trial 402 finished with value: 0.30673820694618315 and parameters: {'lambda_l1': 6.459109331863101e-06, 'lambda_l2': 0.014041273705081268,

In [31]:
#Obtenemos mejor resultado
study.best_params

{'lambda_l1': 1.310525077276578,
 'lambda_l2': 1.6977622299137084e-07,
 'num_leaves': 40,
 'feature_fraction': 0.8974523915259183,
 'bagging_fraction': 0.523553173342081,
 'bagging_freq': 6,
 'min_child_samples': 74}

In [32]:
#Vamos a replicar el resultado de la optimizacion reentrenando el modelo con el mejor conjunto de hiperparametros
#Generamos parametros incluyendo los fijos y la mejor solución que encontro optuna
lgb_params =  {      
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique())} | study.best_params

lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


#Entreno
lgb_model = lgb.train(lgb_params,
                    lgb_train_dataset)

#Muestro matriz de confusion y kappa
display(plot_confusion_matrix(y_test,lgb_model.predict(X_test).argmax(axis=1)))

cohen_kappa_score(y_test,lgb_model.predict(X_test).argmax(axis=1),
                             weights = 'quadratic')


np.float64(0.32051092338594867)

## Modelo con cross validation y conjunto de test

In [33]:
#Genero una metrica para que lightGBM haga la evaluación y pueda hacer early_stopping en el cross validation
def lgb_custom_metric_kappa(dy_pred, dy_true):
    metric_name = 'kappa'
    value = cohen_kappa_score(dy_true.get_label(),dy_pred.argmax(axis=1),weights = 'quadratic')
    is_higher_better = True
    return(metric_name, value, is_higher_better)

#Funcion objetivo a optimizar. En este caso vamos a hacer 5fold cv sobre el conjunto de train. 
# El score de CV es el objetivo a optimizar. Ademas vamos a usar los 5 modelos del CV para estimar el conjunto de test,
# registraremos en optuna las predicciones, matriz de confusion y el score en test.
# CV Score -> Se usa para determinar el rendimiento de los hiperparametros con precision 
# Test Score -> Nos permite testear que esta todo OK, no use (ni debo usar) esos datos para nada en el entrenamiento 
# o la optimizacion de hiperparametros

def cv_es_lgb_objective(trial):

    #PArametros para LightGBM
    lgb_params = {      
                        #PArametros fijos
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique()),
                        #Hiperparametros a optimizar utilizando suggest_float o suggest_int segun el tipo de dato
                        #Se indica el nombre del parametro, valor minimo, valor maximo 
                        #en elgunos casos el parametro log=True para parametros que requieren buscar en esa escala
                        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        } 

    #Voy a generar estimaciones de los 5 modelos del CV sobre los datos test y los acumulo en la matriz scores_ensemble
    scores_ensemble = np.zeros((len(y_test),len(y_train.unique())))

    #Score del 5 fold CV inicializado en 0
    score_folds = 0

    #Numero de splits del CV
    n_splits = 5

    #Objeto para hacer el split estratificado de CV
    skf = StratifiedKFold(n_splits=n_splits)

    for i, (if_index, oof_index) in enumerate(skf.split(X_train, y_train)):
        
        #Dataset in fold (donde entreno) 
        lgb_if_dataset = lgb.Dataset(data=X_train.iloc[if_index],
                                        label=y_train.iloc[if_index],
                                        free_raw_data=False)
        
        #Dataset Out of fold (donde mido la performance del CV)
        lgb_oof_dataset = lgb.Dataset(data=X_train.iloc[oof_index],
                                        label=y_train.iloc[oof_index],
                                        free_raw_data=False)

        #Entreno el modelo
        lgb_model = lgb.train(lgb_params,
                                lgb_if_dataset,
                                valid_sets=lgb_oof_dataset,
                                callbacks=[lgb.early_stopping(10, verbose=False)],
                                feval = lgb_custom_metric_kappa
                                )
        
        #Acumulo los scores (probabilidades) de cada clase para cada uno de los modelos que determino en los folds
        #Se predice el 20% de los datos que separe para tes y no uso para entrenar en ningun fold
        scores_ensemble = scores_ensemble + lgb_model.predict(X_test)
        
        #Score del fold (registros de dataset train que en este fold quedan out of fold)
        score_folds = score_folds + cohen_kappa_score(y_train.iloc[oof_index], 
                                                            lgb_model.predict(X_train.iloc[oof_index]).argmax(axis=1),weights = 'quadratic')/n_splits


    #Guardo prediccion del trial sobre el conjunto de test
    # Genero nombre de archivo
    predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
    # Copia del dataset para guardar la prediccion
    predicted_df = test.copy()
    # Genero columna pred con predicciones sumadas de los 5 folds
    predicted_df['pred'] = [scores_ensemble[p,:] for p in range(scores_ensemble.shape[0])]
    # Grabo dataframe en temp_artifacts
    dump(predicted_df, predicted_filename)
    # Indico a optuna que asocie el archivo generado al trial
    upload_artifact(trial, predicted_filename, artifact_store)    

    #Grabo natriz de confusion
    #Nombre de archivo
    cm_filename = os.path.join(PATH_TO_TEMP_FILES,f'cm_{trial.study.study_name}_{trial.number}.jpg')
    #Grabo archivo
    plot_confusion_matrix(y_test,scores_ensemble.argmax(axis=1)).write_image(cm_filename)
    #Asocio al trial
    upload_artifact(trial, cm_filename, artifact_store)

    #Determino score en conjunto de test y asocio como metrica adicional en optuna
    test_score = cohen_kappa_score(y_test,scores_ensemble.argmax(axis=1),weights = 'quadratic')
    trial.set_user_attr("test_score", test_score)

    #Devuelvo score del 5fold cv a optuna para que optimice en base a eso
    return(score_folds)

In [34]:
#Inicio el store de artefactos (archivos) de optuna
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

#Genero estudio
study = optuna.create_study(direction='maximize',
                            storage="sqlite:///../work/db.sqlite3",  # Specify the storage URL here.
                            study_name="04 - LGB Multiclass CV",
                            load_if_exists = True)
#Corro la optimizacion
study.optimize(cv_es_lgb_objective, n_trials=100)

C:\Users\cacic\AppData\Local\Temp\ipykernel_7360\1746730829.py:2: ExperimentalWarning:

FileSystemArtifactStore is experimental (supported from v3.3.0). The interface can change in the future.

[I 2024-08-22 19:47:04,017] Using an existing study with name '04 - LGB Multiclass CV' instead of creating a new one.
C:\Users\cacic\AppData\Local\Temp\ipykernel_7360\4150719641.py:86: ExperimentalWarning:

upload_artifact is experimental (supported from v3.3.0). The interface can change in the future.

C:\Users\cacic\AppData\Local\Temp\ipykernel_7360\4150719641.py:94: ExperimentalWarning:

upload_artifact is experimental (supported from v3.3.0). The interface can change in the future.

[I 2024-08-22 19:47:22,420] Trial 305 finished with value: 0.35171411148804355 and parameters: {'lambda_l1': 4.332240875388796e-05, 'lambda_l2': 0.06739668969940067, 'num_leaves': 52, 'feature_fraction': 0.4995184362144431, 'bagging_fraction': 0.655096103051531, 'bagging_freq': 7, 'min_child_samples': 14}. Best i

Para ver el optuna dashboard tengo que correr este comando en la terminal

In [49]:
!optuna-dashboard sqlite:///../work/db.sqlite3 --artifact-dir ../work/optuna_artifacts --port 8081

/home/gugui/anaconda3/envs/ldi2/lib/python3.12/site-packages/optuna_dashboard/_cli.py:127: ExperimentalWarning: FileSystemArtifactStore is experimental (supported from v3.3.0). The interface can change in the future.
  artifact_store = FileSystemArtifactStore(args.artifact_dir)
Listening on http://127.0.0.1:8081/
Hit Ctrl-C to quit.

127.0.0.1 - - [21/Aug/2024 17:52:07] "GET / HTTP/1.1" 302 0
127.0.0.1 - - [21/Aug/2024 17:52:07] "GET /static/bundle.js HTTP/1.1" 200 2986981
127.0.0.1 - - [21/Aug/2024 17:52:12] "GET /api/studies HTTP/1.1" 200 276
127.0.0.1 - - [21/Aug/2024 17:52:13] "GET /favicon.ico HTTP/1.1" 200 7670
127.0.0.1 - - [21/Aug/2024 17:52:25] "GET /api/studies HTTP/1.1" 200 276
127.0.0.1 - - [21/Aug/2024 17:52:27] "GET /api/meta HTTP/1.1" 200 62
/home/gugui/anaconda3/envs/ldi2/lib/python3.12/site-packages/optuna_dashboard/_importance.py:75: ExperimentalWarning: PedAnovaImportanceEvaluator is experimental (supported from v3.6.0). The interface can change in the future.
  retu